In [ ]:
# import python packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import pickle
import os
import importlib
from scipy.stats import chi2_contingency
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import itertools
from scipy import stats

In [ ]:
site_list = ['KUMC', 'UTSW', 'MCW', 'UofU', 'UIOWA', 'UMHC', 'UPITT', 'UTHSCSA', 'UNMC']
ext_list = ['csv','dsv', 'dsv', 'csv', 'csv', 'csv', 'csv', 'csv', 'csv']
sep_list = [',','|', '|', '|', ',', ',', ',', ',', '|']
encoding_list = ['utf-8','utf-8','utf-8','utf-8','utf-8','utf-8', 'windows-1252','utf-8', 'utf-16'] 
ct = 0

site = site_list[ct]
ext = ext_list[ct]
sep = sep_list[ct]
encoding = encoding_list[ct]
path = []

if site != 'KUMC':
    rawpath = '/blue/yonghui.wu/hoyinchan/Data/data2022raw/' + site + '/raw/'
else: 
    rawpath = '/blue/yonghui.wu/hoyinchan/Data/data2022raw/' + site + '_ORCALE/raw/'
path.append(rawpath)
path.append('/blue/yonghui.wu/hoyinchan/Data/data2022/' + site + '/')
pdata = '/blue/yonghui.wu/hoyinchan/Data/data2022/'+ site 
site

In [ ]:
def redo_onset(site, pdata):


    onsets = pd.read_pickle(pdata+'/onset00.pkl')
    admit  = try_load_csv('onsets', site, ext, sep, path, admit=None, datecol='ADMIT_DATE')

    # Format datetime
    admit['ADMIT_DATE'] = pd.to_datetime(pd.to_datetime(admit['ADMIT_DATE']).dt.date)
    admit['DISCHARGE_DATE'] = pd.to_datetime(pd.to_datetime(admit['DISCHARGE_DATE']).dt.date)

    # Get back non-aki
    onsets_new = admit[['PATID', 'ENCOUNTERID', 'ADMIT_DATE', 'DISCHARGE_DATE']].merge(onsets , on = ['PATID', 'ENCOUNTERID', 'ADMIT_DATE'] , how='outer')

    # fill stage
    onsets_new['AKI_STAGE'] = onsets_new['AKI_STAGE'].fillna(0).astype(int)

    # use discharge date as since_admit for aki0
    onsets_new['DISCHARGE_SINCE_ADMIT'] = (onsets_new['DISCHARGE_DATE']-onsets_new['ADMIT_DATE']).dt.days
    onsets_new['SINCE_ADMIT'] = np.where(onsets_new['ONSET_SINCE_ADMIT'].notna(), onsets_new['ONSET_SINCE_ADMIT'], onsets_new['DISCHARGE_SINCE_ADMIT'])

    onsets_new.to_parquet(pdata+'/p0_onset_'+site+'.parquet')

In [ ]:
for ct in range(len(site_list)):
    site_list = ['KUMC', 'UTSW', 'MCW', 'UofU', 'UIOWA', 'UMHC', 'UPITT', 'UTHSCSA', 'UNMC']
    ext_list = ['csv','dsv', 'dsv', 'csv', 'csv', 'csv', 'csv', 'csv', 'csv']
    sep_list = [',','|', '|', '|', ',', ',', ',', ',', '|']
    encoding_list = ['utf-8','utf-8','utf-8','utf-8','utf-8','utf-8', 'windows-1252','utf-8', 'utf-16'] 

    site = site_list[ct]
    ext = ext_list[ct]
    sep = sep_list[ct]
    encoding = encoding_list[ct]
    path = []

    if site != 'KUMC':
        rawpath = '/blue/yonghui.wu/hoyinchan/Data/data2022raw/' + site + '/raw/'
    else: 
        rawpath = '/blue/yonghui.wu/hoyinchan/Data/data2022raw/' + site + '_ORCALE/raw/'
    path.append(rawpath)
    path.append('/blue/yonghui.wu/hoyinchan/Data/data2022/' + site + '/')
    pdata = '/blue/yonghui.wu/hoyinchan/Data/data2022/'+ site 
    print(site)
    
    redo_onset(site, pdata)

In [ ]:
def try_load_csv(dataname, site, ext, sep, path, admit, datecol=None):
    
    datadtypes =  {"PATID": 'object', "ENCOUNTERID": 'object', "ONSETS_ENCOUNTERID": 'object', '\ufeff"ONSETS_ENCOUNTERID"': 'object', #General
               "RESULT_NUM":"Float64",  "LAB_LOINC": 'object', "LAB_PX_TYPE": 'object', "RESULT_UNIT": 'object', #lab
               "RESULT_QUAL": 'object', "RESULT_NUM":"Float64", "SPECIMEN_SOURCE": 'object', #lab
               "SEX": 'Int64', "SEX": 'category', "RACE": 'category', "HISPANIC": 'category', #demo, demodeath
               "DX": 'object', "DX_TYPE": 'object', #DX
               "PX": 'object', "PX_TYPE": 'object', #PX
               "MEDADMIN_TYPE": "object", "MEDADMIN_CODE": 'object', #AMED
               "RXNORM_CUI": "object",
               "SYSTOLIC": 'Float64', "DIASTOLIC": 'Float64', "ORIGINAL_BMI": 'Float64', "WT": 'Float64', #VITAL_OLD
               "OBSCLIN_TYPE": "object", "OBSCLIN_CODE": "object"} #VITAL      
    
    print(f"processing : {dataname}")
    filename = 'AKI_'+dataname.upper()
    outfilename = f"/p0_{dataname}_{site}.pkl"    
    
    # Site Specific filenames
    if 'UMHC' in path[0]:
        filename = 'DEID_'+filename    
    if 'UofU' in path[0]:
        if dataname == 'vital':
            filename = 'DEID_AKI_VITAL_OLD'
        if  dataname == 'dx_current':
            filename = 'DEID_AKI_DX_CURRENT_ADMIT_DATE'                
    
    try:
        # Try to load the file from the first path
        df = pd.read_csv( path[0] +  filename + '.' + ext, engine="pyarrow", sep=sep, encoding=encoding, dtype='object')
    except Exception as e:
        print(f"Failed to load from {path[0]}: {e}. Loading converted csv...")
        try:
            # If the first attempt fails, try to load the file from the second path
            #df = pd.read_csv( path[0] +  filename + '.' + ext, engine="pyarrow", sep=sep)
            df = pd.read_csv(path[1] +  filename + '.csv', engine="pyarrow", sep=',', encoding=encoding, dtype='object')
        except Exception as e:
            try:
                # If the first attempt fails, try to load the file from the second path
                df = pd.read_csv(path[1] +  filename + '.csv', sep=',', on_bad_lines = 'skip', encoding=encoding, dtype='object')
            except Exception as e:
                # If the second attempt also fails, handle or re-raise the exception
                print(f"Failed to load from {path[1]} as well: {e}")
                raise Exception(f"Could not load the file from either path.")
        
    try:
        # handle UofU column shifted
        if 'UofU' in path[0] and dataname=='dx':
            df = df.reset_index()
            df = df.drop('RAW_DX_POA',axis=1)
            df.columns = ['ONSETS_ENCOUNTERID', 'DIAGNOSISID', 'PATID', 'ENCOUNTERID',
                   'ENC_TYPE', 'ADMIT_DATE', 'DX_DATE', 'PROVIDERID', 'DX', 'DX_TYPE',
                   'DX_SOURCE', 'DX_ORIGIN', 'PDX', 'DX_POA', 'RAW_DX', 'RAW_DX_TYPE',
                   'RAW_DX_SOURCE', 'RAW_PDX', 'RAW_DX_POA', 'DAYS_SINCE_ADMIT']
            df['DX_DATE'] = df['ADMIT_DATE'] 
        
    
        if 'UofU' in path[0] and dataname=='demo':
            df = df.reset_index()
            df = df.drop(['RAW_HISPANIC', 'RAW_RACE', 'BIOBANK_FLAG'],axis=1)
            df.columns = ['ONSETS_ENCOUNTERID', 'AGE', 'PATID',
                   'BIRTH_DATE', 'BIRTH_TIME', 'SEX', 'SEXUAL_ORIENTATION',
                   'GENDER_IDENTITY', 'HISPANIC', 'BIOBANK_FLAG', 'RACE',
                   'PAT_PREF_LANGUAGE_SPOKEN', 'RAW_SEX', 'RAW_SEXUAL_ORIENTATION',
                   'RAW_GENDER_IDENTITY', 'RAW_HISPANIC', 'RAW_RACE',
                   'RAW_PAT_PREF_LANGUAGE_SPOKEN', 'DEATH_DATE', 'DDAYS_SINCE_ENC',
                   'DEATH_DATE_IMPUTE', 'DEATH_SOURCE']   
        
        # Some site use admit date as dx date
        if 'DX_DATE' in df.columns and df['DX_DATE'].isna():
            df['DX_DATE'] = df['ADMIT_DATE']
                
        df.rename(columns = {'\ufeff"ONSETS_ENCOUNTERID"': 'ONSETS_ENCOUNTERID'}, inplace = True) # to handle the BOM character in UTHSCSA
        df.columns = df.columns.str.upper()
        df.columns = df.columns.str.replace('"+PD.DATE_SHIFT"','').str.replace('AKI.','') # To handle the starnge date name in KUMC
        
        # if not onset
        if admit is not None:
            df["ENCOUNTERID"] = df["ONSETS_ENCOUNTERID"]   
            df = df.drop('ADMIT_DATE', axis=1, errors='ignore')
            df = admit[["PATID","ENCOUNTERID", 'ADMIT_DATE']].merge(df, on = ["PATID","ENCOUNTERID"], how = "inner")

            # recalculate DAYS_SINCE_ADMIT using day as unit
            if datecol is not None:
                df[datecol] = pd.to_datetime(pd.to_datetime(df[datecol]).dt.date)
                df['DAYS_SINCE_ADMIT'] = (df[datecol]-df['ADMIT_DATE']).dt.days
                df = df.drop('ADMIT_DATE',axis=1)
        
        # Convert dataype
        filtered_datadtypes = {key: datadtypes[key] for key in datadtypes if key in df.columns}
        df = df.astype(filtered_datadtypes)            
            
        
        if dataname == 'onsets':
            return df
        else:
            df.to_pickle(pdata+outfilename)
        
    except Exception as e:
        print (f"{dataname} failed at postprocessing")
        raise Exception(f"Could not load the file from either path.")